# In this project, Google is our chosen API

In [334]:
from dotenv import load_dotenv
import os
from urllib.parse import urlencode, urlparse, parse_qsl
import requests
from pymongo import MongoClient
import json

In [335]:
load_dotenv()

True

 (***Very Important Debugging!!! The variable in `.env` SHOULD NOT be inside quotations. The mistake made here was having both exactly the same in `.env` and in `jupyter` as the argument of `os.getenv`.*** )

In [336]:
g_key = os.getenv("GOOGLE_KEY2")

In [337]:
type(g_key)

str

### Formulating API Sample Request - Method 1 
- uses `urlencode(params)`

In [ ]:
g_key = os.getenv("GOOGLE_KEY2")
data_type = "json"
endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
params = {"address":"1600 Amphitheatre Parkway, Mountain View, CA", "key":g_key}


## urlencode puts the parameters in the url from a dictionary
url_params = urlencode(params) #print(url_params)


## We used this sample to construct the above parameters
#sample = "https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY"

url = f"{endpoint}?{url_params}"

#### Sample function for Request - Method 1


In [201]:
g_key = os.getenv("GOOGLE_KEY2")
def extract_lat_lng(address_or_postalcode, data_type = "json"):
    endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": address_or_postalcode, "key":g_key}
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return {}
    latlng = {}
    try:
         latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"),latlng.get("lng")

In [202]:
extract_lat_lng("1600 Amphitheatre Parkway, Mountain View, CA")

(37.42165019999999, -122.0856843)

In [ ]:
https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
data_type = "json"
endpoint = f"https://maps.googleapis.com/maps/api/place/findplacefromtex/{data_type}"   
params = {"findplacefromtext" = "starbucks"}    
url_params = urlencode(params)   
url = f"{endpoint}?{url_params}"

### Forming Sample API Request - Method 2
- uses `endpoint.format(**params))`

In [251]:
g_key = os.getenv("GOOGLE_KEY2")

In [252]:
data_type = "json"
url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/{data_type}"


In [261]:
endpoint = "{input}&{inputtype}&{fields}&{locationbias}&{key}"
params = {
    "input":"starbucks",
    "inputtype":"textquery",
    "fields":"formatted_address,name,geometry",
    "locationbias":"point:lat,lng",
    "key": f"{g_key}",
}

data = requests.get(url+endpoint.format(**params)) 

In [262]:
print(data)

<Response [404]>


### Sample API Requests for Place
- parameters are separated for clarity

In [ ]:
https://maps.googleapis.com/maps/api/place/findplacefromtext/json?
    input=Museum%20of%20Contemporary%20Art%20Australia&
    inputtype=textquery&
    fields=photos,formatted_address,name,rating,opening_hours,geometry&
    key=YOUR_API_KEY
https://maps.googleapis.com/maps/api/place/findplacefromtext/json?
    input=%2B61293744000&
    inputtype=phonenumber&
    fields=place_id&
    key=YOUR_API_KEY
https://maps.googleapis.com/maps/api/place/findplacefromtext/
    json?input=mongolian%20grill&
    inputtype=textquery&
    fields=photos,formatted_address,name,opening_hours,rating&
    locationbias=circle:2000@47.6918452,-122.2226413&
    key=YOUR_API_KEY

                 

### Decision: Stay with Method 1
- **IMPORTANT** pass *url* to `request.get` to hide key

In [338]:
data_type = "json"
endpoint = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/{data_type}"
params = {
    
    "input":"starbucks",
    "inputtype":"textquery",
    "fields":"formatted_address,name,geometry",
    "locationbias":"point:lat,lng",
    "key": f"{g_key}"
}


url_params = urlencode(params)
url = f"{endpoint}?{url_params}"

In [339]:
data = requests.get(url)

In [285]:
data

<Response [200]>

In [286]:
type(data)

requests.models.Response

In [287]:
data = data.json()

In [288]:
type(data)

dict

In [289]:
print(data)

{'candidates': [{'formatted_address': 'Calle del Dr. Drumen, 4, 28012 Madrid, España', 'geometry': {'location': {'lat': 40.408792, 'lng': -3.6930134}, 'viewport': {'northeast': {'lat': 40.41017507989272, 'lng': -3.691729070107278}, 'southwest': {'lat': 40.40747542010728, 'lng': -3.694428729892722}}}, 'name': 'Starbucks'}], 'status': 'OK'}


### We can see our Method 1 has produced a result, but the result is actually near our currrent location
- change plan to use **Google nearby search** to define a location such as one of our select cities

In [ ]:
# sample nearby search request with "*" next to required fields
https://maps.googleapis.com/maps/api/place/nearbysearch/json?
  * location=-33.8670522,151.1957362&
  * radius=1500&
    type=restaurant&
    keyword=cruise&
  * key=YOUR_API_KEY

### The following function will take lat and long of a given locale, the radius we want to search, and what we want nearby

In [356]:
def nearby_search(lat, long, radius_in_meters, keyword):
    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {

        "location":f"{lat},{long}",
        "radius":f"{radius_in_meters}",
        "rankby" : "<distance - must include keyword,name,or type",
        "keyword":f"{keyword}",
        "inputtype":"textquery",
        "fields":"formatted_address,name,geometry",
        "key": f"{g_key}"
    }


    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    data = requests.get(url)
    return data.json()

## Let's give it a try 🤞
****Spoiler: it didn't work so we went back to previous function for some debuggin'*** 🐞

### We are using the first latitude and longitude from our list LA locations further down below.

In [351]:
nearby_search(34.057498,-118.446596, 500, "starbucks")

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [ ]:
endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {

    "location":"34.057498,-118.446596",
    "radius":"500",
    "keyword":"starbucks",
    "inputtype":"textquery",
    "fields":"formatted_address,name,geometry",
    "key": f"{g_key}"
}

url_params = urlencode(params)
url = f"{endpoint}?{url_params}"
data = requests.get(url)
#return data.json()
starbucks = data.json()

In [363]:
starbucks = starbucks["results"]

In [365]:
len(starbucks)

5

### Well, aren't Execs gonna be giddy about this one!!! There's a Starbucks at every corner within a 500m radius!

In [383]:
starbucks_geometries =[] 

for location in starbucks:
    lat_lng = []
    lat_lng.append(location["geometry"]['location']['lat'])
    lat_lng.append(location["geometry"]['location']['lng'])
    starbucks_geometries.append(lat_lng)
    
    
print(starbucks_geometries)    

[[34.0594359, -118.4447505], [34.062529, -118.447346], [34.062686, -118.444064], [34.0540732, -118.4404088], [34.0598259, -118.4445406]]


# We are going to look for vegan restaruants in Los Angeles...

In [411]:
endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {

    "location":"34.057498,-118.446596",
    "radius":"200",
    "keyword":"vegan restaurants",
    "inputtype":"textquery",
    "fields":"formatted_address,name,geometry",
    "key": f"{g_key}"
}

url_params = urlencode(params)
url = f"{endpoint}?{url_params}"
data = requests.get(url)
#return data.json()
vegan = data.json()

In [412]:
vegan = vegan["results"]


In [413]:
len(vegan)

9

In [414]:
vegan_geometries =[] 

for location in vegan:
    lat_lng = []
    lat_lng.append(location["geometry"]['location']['lat'])
    lat_lng.append(location["geometry"]['location']['lng'])
    vegan_geometries.append(lat_lng)
    
    
print(vegan_geometries)    

[[34.0593399, -118.4451485], [34.0598901, -118.4459704], [34.0597447, -118.4456233], [34.0572508, -118.4431379], [34.059878, -118.444609], [34.0604948, -118.4445603], [34.06111509999999, -118.4462877], [34.061434, -118.446446], [34.061132, -118.445682]]


## ***Looks like our vegan bet paid off !!***

In [340]:
client = MongoClient("mongodb://localhost")
db = client.get_database("companydb")
comp_find = db.companies.find({})
comp_find = list(comp_find)
len(comp_find)

18801

In [341]:
comp = db.companies

In [326]:
res = comp.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$match":{"offices.city":{"$ne":None}}},
                      {"$match":{"offices.city":{"$ne": ""}}},
                      {"$match":{"offices.city": "Los Angeles"}},
                      {"$project":{"_id":0}}
                    ])


In [325]:
len(list(res))

168

In [298]:
db.los_angeles_offices.insert_many(res)

# Now we have our collection of possible offices in L.A.

In [342]:
filter_q = { }
project = {"_id":0, "offices.latitude":1 ,"offices.longitude":1 }
res = list(db.los_angeles_offices.find(filter_q, project))

In [347]:
print(res[0]['offices']['latitude'],res[0]['offices']['longitude'])

34.057498 -118.446596


In [308]:
for site in res:
    lat = site['offices']['latitude']
    lon = site['offices']['longitude']
    

In [331]:
db = client.get_database("datamad1020")
coll = db.los_angeles_offices

In [332]:
res = coll.create_index([("coord", "2dsphere")])

In [333]:
res

'coord_2dsphere'

## Let's . . . Get . . . VI-SU-AL . . . 

![Olivia Newton](olivianewton.jpg "Physical!")

(Go to [geoJSON and Folium](/notebooks/geoJSON_and_Folium.ipynb) next)